# Cvičenie 5: Wordle

Wordle je jednoduchá webová hra, ktorá sa stala populárnou v posledných rokoch. Cieľom hry je uhádnuť päťpísmenové slovo na šesť pokusov, pričom hra po každom pokuse prezradí, či náš tip obsahuje správne písmená, a či tie sú na správnom mieste. Na základe tejto spätnej väzby vieme eliminovať možné riešenia, a tak sa dopracovať ku skrytému slovu. [Ak s hrou ešte nemáte skúsenosti, môžete si ju vyskúšať tu.](https://wordlegame.org)

<img src="sources/lab05/wordle_example.png" width="600">

<p style="text-align: center;">Zdroj: https://en.wikipedia.org/wiki/Wordle#/media/File:Wordle_196_example.svg</p>

Príklad hry môžete vidieť na obrázku vyššie. Skryté slovo je *rebus*. Prvý pokus hráča bol *arise*. Farby jednotlivých písmen reprezentujú informácie o správnosti písmen a ich pozícií. Napríklad pri slove *arise* sa písmená *r*, *s* a *e* nachádzajú aj v správnom riešení *rebus*, nie sú ale na správnej pozícii. Zvyšné sivé písmená sa v správnom riešení nenachádzajú. V ďalšom pokuse hráč zadal slovo *route*, v ktorom písmeno *r* je už na správnej pozícii a ďalej sa dozvedel aj to, že riešenie obsahuje aj písmeno *u*. Takýmto štýlom hráč pokračuje, až kým nenájde správne slovo, alebo nevyužije všetkých šesť pokusov. V štandardnej verzii hry hráč môže zadať ľubovoľné slovo zo sady známych slov. Ak hráč zadá neplatné slovo, môže pokračovať bez toho, aby prišiel o pokus.

Pred začatím implementácie si potrebujete stiahnuť [súbor so zoznamom akceptovaných slov](sources/lab05/word_list.txt). [Následne si môžete stiahnuť kostru riešenia](sources/lab05/lab05.py), alebo pracovať v tomto notebooku.

## 1. Načítanie slov

Základom hry bude súbor so sadou prijateľných slov. Príklad pre takýto súbor nájdete vyššie (`word_list.txt`): každý riadok obsahuje jedno slovo a na konci súboru sa nachádza prázdny riadok.

V prvom kroku implementujete funkciu `load_words`, ktorá načíta zoznam slov akceptovaných hrou. Funkcia má jeden parameter (`dataset_path`) – cestu k súboru, v ktorom sa nachádzajú podporované slová. Funkcia vracia zoznam slov, pričom jednotlivé prvky zoznamu sú reťazce s dĺžkou 5.

In [13]:
import pandas as pd

In [14]:
def load_words(dataset_path):
    try:
        data = pd.read_csv(dataset_path, engine='python', sep=" ", header=None)[0].to_list()
        return data
    except:
        return None


## 2. Začatie a ukončenie hry

K samotnej hre potrebujeme ďalšie dve jednoduché pomocné funkcie. Najprv implementujte funkciu `get_puzzle`, ktorá zo zoznamu slov, ktorý dostane ako parameter `word_list`, vyberie náhodné slovo, ktoré bude slúžiť ako hľadané riešenie. Funkcia má jednu návratovú hodnotu: slovo, teda string s dĺžkou päť. Pre náhodný výber použite štandardný modul `random`, ktorý už máte naimportovaný.

Dôležitou funkcionalitou je zabezpečiť, aby sme zistili, či hráč už uhádol správne riešenie. K tomu slúži funkcia `is_game_finished` s dvoma parametrami:
 - `guess` – reťazec reprezentujúci tip hráča;
 - `puzzle` – reťazec reprezentujúci správne riešenie.
Funkcia vracia hodnotu `True`, ak hráč slovo uhádol, v opačnom prípade vráti `False`.

**Poznámka:** Nezabudnite, že hráč môže svoj tip napísať aj veľkými písmenami, v takomto prípade ale musíte jeho tip vyhodnotiť takisto správne.

In [15]:
import random

def get_puzzle(word_list):
    return word_list[random.randrange(len(word_list))]

def is_game_finished(guess, puzzle):
    return guess.lower() == puzzle


## 3. Vyhodnotenie tipu

Poslednou funkciou k jednoduchej implementácii je `evaluate_guess`, ktorá vyhodnotí tip hráča a vráti mu informácie o správnosti písmen a ich pozícií. Funkcia má rovnaké parametre ako `is_game_finished`, teda tip hráča (`guess`) a správne riešenie (`puzzle`). Návratová hodnote je ale trošku zložitejšia – bude to zoznam n-tíc (*list of tuples*), ktorý reprezentuje spätnú väzbu po vyhodnotení tipu a to tak, že pre každé písmeno sa vytvorí trojica hodnôt:
 - písmeno – reťazec s dĺžkou 1
 - informácia o tom, či sa dané písmeno nachádza v riešení – booleovská hodnota `True/False`
 - informácia o tom, či dané písmeno je na správnej pozícii – booleovská hodnota `True/False`; ak sa písmeno v riešení nenachádza, bude `False`, keďže písmeno je na nesprávnej pozícii.

V našom príklade teda tip route by sme vyhodnotili nasledovne:
`[('r', True, True), ('o', False, False), ('u', True, False), ('t', False, False), ('e', True, False)]`

Ak hráč uhádne správne slovo, všetky booleovské hodnoty budú `True`, ak zadá slovo, ktoré nemá žiadne písmeno spoločné s riešením, všetky hodnoty budú `False`.

**Poznámka:** Pri príprave vyhodnotenia musíte dodržujte poradie písmen v slove. Zatiaľ neriešte opakujúce sa písmená.

In [16]:
def evaluate_guess(guess, puzzle):
    final_list = []
    for index, guessChar in enumerate(list(guess)):
        found = False
        position = False

        if guessChar in list(puzzle):
            found = True
            if list(puzzle)[index] == guessChar:
                position = True
        final_list.append((guessChar, found, position))

    return final_list

print(evaluate_guess('data', 'data'))

[('d', True, True), ('a', True, True), ('t', True, True), ('a', True, True)]


## 4. Zahrajme si Wordle

Teraz už máme implementované všetky podporné funkcie, vieme sa teda spustiť do implementácie hry Wordle. Zadefinujte teda funkciu `start_game` s parametrom `dataset_path`, ktorá zastrešuje celú hru a to nasledovne:

1. načíta zoznam povolených slov zo súboru, ktorý dostane v parametri;
2. vyberie náhodné slovo zo zoznamu, ktoré bude slúžiť ako hľadané riešenie;
3. vykoná sa maximálne šesť kôl, pričom v každom kole:

    1. používateľ môže zadať svoj tip - ak tip nie je zo zoznamu povolených slov, hráč je opätovne vyzvaný na zadanie svojho tipu bez toho, aby prišiel o možnosť tipnúť si (neráta sa to do šiestich pokusov);
    2. vyhodnotí sa hráčov tip a výsledok sa vypíše na obrazovku (iba vypíšte programovú reprezentáciu, nepotrebujete ju nijak upravovať);
    3. ak hráč uhádne slovo, vypíšte o tom správu a ukončite hru; ak hráč minul všetky možnosti ale slovo neodhalil, informujte ho o prehre, a hru tiež ukončite.
    
**Poznámka:** Pre jednoduchosť testovania môžete vypísať hľadané slovo hneď na začiatku, aby ste mohli overiť správnosť podporných funkcií.

In [17]:
def start_game(dataset_path):
    try:
        words = load_words(dataset_path)
        if words is None:
            print('Error loading dataset - Wrong dataset path.')
            return None
        chosen_word = get_puzzle(words)
        rounds = 0
        won = False
        while rounds < 6 and not won:
            guess = input('Whats your guess: ')
            if len(guess) != 5:
                print('Guess needs to be 5 characters')
                continue

            if is_game_finished(guess, chosen_word):
                print('Congratulations, you won with guess {}'.format(guess))
                won = True
                break

            print(evaluate_guess(guess, chosen_word))
            rounds += 1

        if not won:
            print('Sorry, you lost. The word was {}!'.format(chosen_word))
    except:
        print("Something went wrong! Try again later.")
        return None



start_game("sources/lab05/word_list.txt");

[('a', True, False), ('b', False, False), ('o', False, False), ('v', False, False), ('e', False, False)]
[('s', False, False), ('p', False, False), ('e', False, False), ('n', False, False), ('d', False, False)]
[('c', True, True), ('l', False, False), ('e', False, False), ('a', True, False), ('n', False, False)]
[('s', False, False), ('m', False, False), ('a', True, False), ('l', False, False), ('l', False, False)]
[('c', True, True), ('a', True, False), ('r', True, True), ('r', True, False), ('y', False, False)]
[('c', True, True), ('a', True, False), ('t', False, False), ('c', True, True), ('h', False, False)]
Sorry, you lost. The word was circa!


## 5. Ošetrenie riešenia

Vaša implementácia je bez pochýb perfektne funkčná, avšak aj najlepší kód môže zlyhať, ak mu dáte nesprávne vstupy a hodnoty na spracovanie. Upravte svoje riešenie tak, aby v žiadnom prípade nekončil chybou. Možné chyby ošetrite pomocou bloku `try-except`, v chybných prípadoch používateľovi poskytnite informatívnu správu o tom, čo sa pokazilo, a program ukončite riadne (teda program nesmie padať).

**Prediskutujte jednotlivé funkcie, predpoklady na vstup a výstup, validujte všetky vstupy pre správnosť typu a hodnôt, a ak používate štandardné funkcie, zistite, za akých okolností fungujú správne.**

## Doplňujúce úlohy

Je známym faktom, že programátori sú leniví, a aj keď je Wordle zábavná hra a skvelý spôsob na prokrastináciu, skôr či neskôr im napadne implementovať inteligentného bota, ktorý by túto hru zahral za nich a omnoho efektívnejšie. V tejto úlohe vytvoríte jedného takéhoto bota.

Prvým krokom je implementovať funkciu `get_player_guess`, ktorá zo zoznamu slov `word_list` (formát rovnaký ako po načítaní funkciou `load_words`), ktorý dostane ako parameter, vyberie zoznam možných riešení a náhodný tip. Funkcia má teda dve návratové hodnoty:
 - zoznam možných riešení – zoznam reťazcov, kde každý reťazec je päťpísmenové slovo; nepracujte priamo s parametrom `word_list`, ale vlastným zoznamom alebo kópiou `word_list`u.
 - tip – reťazec, teda náhodné slovo zo zoznamu možných riešení.

Bot eliminuje možnosti na základe získaných vedomostí o správnom riešení, ktoré sú reprezentované v zozname `knowledge`, ktorý dostane ako parameter. Tento zoznam je inicializovaný na začiatku hry a je to zoznam trojíc, kde každá trojica reprezentuje informáciu o jednotlivých možných písmenách podobne ako pri funkcii `evaluate_guess`.

Trojica má nasledovnú štruktúru:
 - písmeno – string s dĺžkou 1
 - informácia o tom, či sa písmeno nachádza v slove – inicializovaná na `None`, neskôr nahradíte hodnotu booleovskými hodnotami `True/False`
 - informácia o pozícii písmena v správnom riešení – inicializovaná na -1, neskôr nahradíte platným indexom 0 až 4. Poznámka: tento bot nebude rátať s možnosťou viacnásobného výskytu písmena v slove, teda tretia hodnota bude vždy iba jedno číslo.

Na základe týchto vedomostí by mala funkcia `get_player_guess` eliminovať možnosti nasledovne:
 1. vymazať zo zoznamu možných riešení slová, ktoré neobsahujú písmená, o ktorých vie, že sa nachádzajú v správnom riešení na základe zoznamu `knowledge`
 2. vymazať zo zoznamu možných riešení slová, ktoré obsahujú písmená, o ktorých vie, že sa nenachádzajú v správnom riešení na základe zoznamu `knowledge`
 3. vymazať zo zoznamu možných riešení slová, ktoré nemajú správne písmeno na niektorej pozícii, o ktorej už vie, aké tam bude písmeno.

Pre jednoduchosť uvedieme aj jeden príklad. Po prvých dvoch pokusoch uvedených v príklade vyššie hráč už vie, že riešenie určite obsahuje písmená *r*, *s*, *e*, *u* a na prvej pozícii je písmeno *r*. Na základe jednotlivých pravidiel by teda eliminoval napríklad:
 1. slovo *table*, ktoré neobsahuje potrebné písmená *r*, *s*, *u*
 2. slovo *braid*, ktoré obsahuje písmeno *a*, o ktorom už hráč vie, že sa v riešení nenachádza
 3. slovo *trout*, keďže hráč už vie, že na prvej pozícii musí byť písmeno *r*

Po eliminácii niektorých možných riešení, bot vyberie náhodné slovo zo zoznamu a vráti hodnoty podľa špecifikácie funkcie.

Samozrejme bot má šancu hru vyhrať iba vtedy, ak bude postupne aktualizovať reprezentáciu svojich vedomostí o hľadanom slove. K tomu slúži funkcia `process_result` s parametrom `result`, ktorý obsahuje spätnú väzbu od hry, ktorá je vygenerovaná funkciou `evaluate_guess` (formát sa nemení). Funkcia postupne spracuje informácie o všetkých písmenách z posledného tipu a to nasledovne:
 1. aktualizuje informáciu o tom, či sa písmeno nachádza alebo nenachádza v správnom riešení
 2. ak je písmeno aj na správnej pozícii, aktualizuje tento údaj v zozname `knowledge`.

Funkcia `process_result` nemá návratovú hodnotu, aktualizujte priamo zoznam `knowledge`.

**Poznámka:** Nezabudnite, že zoznam knowledge obsahuje n-tice, ktoré sú nemenné. Práve preto pri spracovaní spätnej väzby musíte vytvoriť novú n-ticu a uložiť ju na správne miesto v zozname knowledge. Poradie informácií o písmenách v zozname má pritom ostať rovnaké, t.j. najprv budete mať vedomosti o výskyte písmena *a*, písmena *b*, atď.

Po implementovaní týchto dvoch funkcií máte bota hotového, môžete jeho funkčnosť vyskúšať pomocou nasimulovanej hry. Funkcia je veľmi podobná funkcii `start_game` – vstup od hráča je ale nahradený vstupom od bota. Počas hry sa takisto vypíše zoznam možných riešení, ktorý bude stále kratší ako bot spracuje informácie, ktoré sa dozvie o hľadanom slove.

In [18]:
from copy import deepcopy
import string


def get_player_guess(word_list, knowledge):
    available = word_list.copy()

    # TODO: eliminate impossible solutions

    return available, random.choice(available)


def process_result(result, knowledge):
    # TODO: update knowledge based on result
    pass


def bot_game(dataset_path):
    word_list = load_words(dataset_path)
    player_words = word_list.copy()

    PLAYER_KNOWLEDGE = [(letter, None, -1)
                        for letter in string.ascii_lowercase]
    player_knowledge = deepcopy(PLAYER_KNOWLEDGE)

    puzzle = get_puzzle(word_list)
    print(puzzle)

    guess = ""
    while not is_game_finished(guess, puzzle):
        player_words, guess = get_player_guess(player_words, player_knowledge)
        print(player_words)
        player_words.remove(guess)
        print(guess)

        result = evaluate_guess(guess, puzzle)
        print(result)

        process_result(result, player_knowledge)


bot_game("word_list.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'word_list.txt'